# Importing the Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Import the Dataset 

In [2]:
Dataset = pd.read_csv(r"C:\Users\Rupankar\Desktop\Deep Learning\Deep_Learning_A_Z\Volume 1 - Supervised Deep Learning\Part 1 - Artificial Neural Networks (ANN)\Section 4 - Building an ANN\Artificial_Neural_Networks\Churn_Modelling.csv")


In [3]:
X= Dataset.iloc[: , 3:13 ].values
Y=Dataset.iloc[: , -1].values

# Encoding the Categorical Values

In [4]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder_X_1 = LabelEncoder()
X[: , 1]= label_encoder_X_1.fit_transform(X[:, 1])
label_encoder_X_2= LabelEncoder()
X[:, 2]=label_encoder_X_2.fit_transform(X[:, 2])
onehot_encoder=OneHotEncoder(categorical_features=[1])
X= onehot_encoder.fit_transform(X).toarray()
X=X[:, 1:]

C:\Users\Rupankar\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Rupankar\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Data Preprocessing

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train,Y_test= train_test_split(X,Y,test_size=0.2,random_state=1)

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test=sc.transform(X_test)

# Importing Keras Libraries and Packages

In [7]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# Creating the ANN Model

In [8]:
model = Sequential()

# Adding the input layer and the first hidden layer

In [9]:
model.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))

# Adding the second hidden layer

In [10]:
model.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding the output layer

In [11]:
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compiling the Model

In [12]:
model.compile(optimizer='adam' , loss= 'binary_crossentropy', metrics=['accuracy'])

# Fitting the Model to the Training Dataset

In [13]:
model.fit(X_train,Y_train, batch_size=10 , epochs= 100)

Epoch 1/100
8000/8000 [==============================] - 5s 654us/step - loss: 0.4845 - accuracy: 0.7970
Epoch 2/100
8000/8000 [==============================] - 1s 175us/step - loss: 0.4285 - accuracy: 0.7972
Epoch 3/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4229 - accuracy: 0.7972
Epoch 4/100
8000/8000 [==============================] - 1s 169us/step - loss: 0.4186 - accuracy: 0.8221
Epoch 5/100
8000/8000 [==============================] - 4s 492us/step - loss: 0.4168 - accuracy: 0.8275
Epoch 6/100
8000/8000 [==============================] - 2s 227us/step - loss: 0.4148 - accuracy: 0.8304
Epoch 7/100
8000/8000 [==============================] - 4s 509us/step - loss: 0.4130 - accuracy: 0.8317
Epoch 8/100
8000/8000 [==============================] - 2s 272us/step - loss: 0.4121 - accuracy: 0.8325
Epoch 9/100
8000/8000 [==============================] - 4s 519us/step - loss: 0.4114 - accuracy: 0.8345
Epoch 10/100
8000/8000 [==============================]

# Predicting the Test Dataset

In [15]:
Y_pred= model.predict(X_test)

In [17]:
Y_pred = (Y_pred>0.5)

In [19]:
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_pred,Y_test)

In [20]:
cm

array([[1539,  283],
       [  46,  132]], dtype=int64)

# Appling the K-Fold Cross Validation

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Building the Classifier Function

In [10]:
def build_classifier():
    model.add(Dense(units=6 ,kernel_initializer='uniform', activation='relu', input_dim=11))
    model.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
    return model


# Applying the K-Fold Cross Validation Algo

In [11]:
classifier=KerasClassifier(build_fn= build_classifier ,batch_size=10 , epochs= 100)
accuracy=cross_val_score(estimator=classifier,X= X_train, y= Y_train, cv= 10, n_jobs= -1)

In [16]:
mean=accuracy.mean()

In [17]:
print(mean)

0.8412500023841858


# Tunning the ANN

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
def build_classifier(optimizer):
    model.add(Dense(units=6 ,kernel_initializer='uniform', activation='relu', input_dim=11))
    model.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(optimizer=optimizer ,loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [14]:
classifier = KerasClassifier(build_fn= build_classifier)

In [15]:
parameter={'batch_size' : [32,64],
           'epochs' : [100,500],
           'optimizer' : ['adam', 'rmsprop']
          }

In [ ]:
grid_search = GridSearchCV(estimator= classifier, param_grid= parameter, scoring= 'accuracy', cv=10)
grid_search=grid_search.fit(X_train,Y_train)
best_parameter = grid_search.best_params_
best_accuracy=grid_search.best_score_